<h1> Introduction </h1>
> This project is on a twitter account - We rate dogs. In this workspace, I gather, assess and clean my data which I will gather from multiple sources.

In [1]:
#import all packages used for the project here
import pandas as pd
import tweepy
import numpy as np
import requests
import json
from requests import Session
import datetime

<h1> Gathering Data </h1>
> In this process we will gather data from three sources. The first is a csv file that is provided to us so we have to upload it in same folder as jupyter notebook .ipnyb file and then load it using pandas as a dataframe for analysis

In [4]:
# load the first csv file first in the same folder as this jupyter notebook and then into a datframe using pandas
twitdf=pd.read_csv('twitter-archive-enhanced.csv')
# to visually analyse it we could use a df.head command as below
twitdf.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


> Next lets load the second data by doing te required work to handle it. We have been provided with a url and requests library

In [3]:
url ="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)
# read the response line by line into the file
filename = 'image-predictions.tsv'
with open(filename, 'wb') as fd:
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)
# store the response from the file into a dataframe
image_df=pd.read_csv('image-predictions.tsv', sep='\t')
image_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


Next, using the tweet IDs in the WeRateDogs Twitter archive, we query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file

In [4]:
# Your authentication data here
consumer_key = "t4YqRJHYUJLYQe6W1ilQwLrXK"
consumer_secret = "5tuofaKpr51eJuiINmTpaYijAxf729pMmY1MSjd3DR4sPBy9oE"
access_token = "130734301-SuJ8O2mHC0WDGkqdmTkvMGjgBmUWCRtJVlANCCVK"
access_token_secret = "7zfrgxv8xPvwuEjvjsa6AJsLx9SdSqdacETpZA1VsXHAs"


In [19]:
# connect to tweepy using auth api and authentication data from above
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser = tweepy.parsers.JSONParser(), # Parse the result to Json Object
                 wait_on_rate_limit = True, # Automatically wait for rate limits to replenish
                 wait_on_rate_limit_notify = True) # Print a notification when Tweepy is waiting for rate limits to replenish
# store all the data in a list of dictionary
list_api=[]
# store tweet id of any error in a list
error_list = []

for tweet_id in twitdf['tweet_id']:
    try:
        page = api.get_status(tweet_id, tweet_mode = "extended")
        retwt_count = json.dumps(page['retweet_count'])
        fav_count =json.dumps(page['favorite_count'])
        list_api.append({'tweet_id': int(tweet_id),'favorite_count': int(fav_count), 'retweet_count': int(retwt_count)})
    except:
        print(str(tweet_id)+'_'+'error')
        error_list.append(tweet_id)

888202515573088257_error
873697596434513921_error
869988702071779329_error
866816280283807744_error
861769973181624320_error
845459076796616705_error
842892208864923648_error
837012587749474308_error
827228250799742977_error
802247111496568832_error
775096608509886464_error
Rate limit reached. Sleeping for: 723


In [20]:
print(len(list_api))
print(len(error_list))

2345
11


> So we were not able to obtain the favorite and retweet information of 11 tweets this way. This list is stored in error_list in case we want to access it at a later stage

In [23]:
# store the list info in a dataframe
api_tweets = pd.DataFrame(list_api)
# Save the dataFrame in file tweet_json.txt
api_tweets.to_csv('tweet_json.csv', encoding = 'utf-8', index=False)
api_tweets

,favorite_count,retweet_count,tweet_id
0,38826,8594,892420643555336193
1,33259,6316,892177421306343426
2,25056,4192,891815181378084864
3,42206,8707,891689557279858688
4,40353,9480,891327558926688256
5,20239,3137,891087950875897856
6,11863,2088,890971913173991426
7,65598,19056,890729181411237888
8,27786,4301,890609185150312448
9,31963,7475,890240255349198849


In [5]:
# read the file that we just stored
api_tweets=pd.read_csv('tweet_json.csv')
api_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
favorite_count    2345 non-null int64
retweet_count     2345 non-null int64
tweet_id          2345 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


<h1> Gathering: Summary </h1>
> Data was essentially gathered from an online link, a csv file and from an online website through api. Now lets proceed to the next step which is data assessing stage
<h1> Assessing Stage </h1>
> Data files gathered need to be inspected both visually and programatically and all quality and tidiness issues need to be enumerated at this step 

In [6]:
# let us first explore twitdf dataframe
twitdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

> Looking at this info, it is clear that some of the tweets are retweets. 181 of them. These will not be used in our analysis since we will only be dealing with original ratings. Dog stage is written in multiple columns whereas one line should be enough for the same. Datatype is not right for timestamp (should be datetime), retweeetd_status_id, retweeted_status_user_id and retweeted_status_timestamp. However the retweeted columns will not be used in our analysis so I am going to get rid of them anyway so I will not change their datatypes. Lets do more analysis of this dataframe to understand what else needs to be done.

In [6]:
twitdf['floofer'].value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [7]:
twitdf['doggo'].value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [8]:
twitdf['puppo'].value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [9]:
twitdf['pupper'].value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

There is no need of 4 columns of the dogtype when it can essentially be told in one column. This is a tidyness issue. Additionally we do not know a dog type for all the dogs. 

In [10]:
twitdf['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

> The above analysis shows that though the rating should have been 10, it is not always the case indicating that maybe they were wrongly extracted from the dataset. We will extract the ratings again just to be sure that they were rightly extracted

In [7]:
# there is a column called source. Lets understand its contents more
twitdf['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

This column has some info that wont be needed and can be deleted. So we will clean up this column.

In [11]:
# lets also read some random columns whose denominator is not 10 or rather all columns whose denominator is not 10.
twitdf[twitdf['rating_denominator'] !=10 ]['text']

313     @jonnysun @Lin_Manuel ok jomny I know you're e...
342              @docmisterio account started on 11/15/15
433     The floofs have been released I repeat the flo...
516     Meet Sam. She smiles 24/7 &amp; secretly aspir...
784     RT @dog_rates: After so many requests, this is...
902     Why does this never happen at my front door......
1068    After so many requests, this is Bretagne. She ...
1120    Say hello to this unbelievably well behaved sq...
1165    Happy 4/20 from the squad! 13/10 for all https...
1202    This is Bluebert. He just saw that both #Final...
1228    Happy Saturday here's 9 puppers on a bench. 99...
1254    Here's a brigade of puppers. All look very pre...
1274    From left to right:\nCletus, Jerome, Alejandro...
1351    Here is a whole flock of puppers.  60/50 I'll ...
1433    Happy Wednesday here's a bucket of pups. 44/40...
1598    Yes I do realize a rating of 4/20 would've bee...
1634    Two sneaky puppers were not initially seen, mo...
1635    Someon

Reading the text of those columns shows that these entries have been misread coz they contain multiple digits or they dont have ratings.

In [7]:
# now, lets look at other column
image_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


> We do not have images for all 2356 entries in twitdf dataframe. There are 3 predictions but I am going to keep only 1 for my analysis. Addiitonally I am going to keep the data on its confidence level and p1_dog which tells whether it is a dog or not for later. I am going to delete everything else. Also, the column names need to be more explanatory than what they are here.

Now for the last dataframe, since we downloaded it ourself, we know there are 11 missing entries for favorites and retweets. All other data is present.

<h2> Quality Issues </h2>
> 1. There are a few Tweets with no images. I am going to delete those rows to have data with all the information even from my other dataframes.
2. Dataset contains 181 retweets which can be seen from the twitdf frame that need to be deleted
3. Dog rating might be incorrect and it needs to be verified from the text provided as some denominator ratings are more than 10. 
4. Eventually 2 columns are not needed for ratings as all will have denominator 10 and we can then name rating_numerator as just rating
5. Incorrect/Missing dog names - but do we really need to tidy this up? I will not be using this in my analysis so I am skipping this step
6. Column names need to be edited to make them self-explanatory. 
7. Tweet id is not an integer and can be made string type
8. Datatypes are not correct (timestamp needs to be converted to datetime and then made in a format one will use for analysis)
9. twitdf dataframe's column, source has a lot of extra information that can be deleted so it makes more sense 


<h2> Tidiness Issues </h2>
>1. Dog stage is written in 4 columns and they can be assembled into one stage by using pandas melt function
2. The three dataframes can be joined into one and then saved as a masterfile 
3. A lot of extra columns such as in the image_df where we dont need prediction 2 and 3 and in_reply_to_status_id, in_reply_to_user_id which can finally be deleted after using them

> Let us start dealing with the issues but to start with. Let us first merge all 3 dataframes into one. For this, I am going to  use tweet_id as the pivot

In [20]:
masterdf= twitdf.join(api_tweets.set_index("tweet_id"), on = "tweet_id", how = "inner")

Now, lets merge the image_df and then I will drop the extraneous columns in the dataframes. For this one, since not aving an image should not be an issue, I am goignt o performa  left join 

In [21]:
masterdf = masterdf.join(image_df.set_index("tweet_id"), on = "tweet_id", how = "inner")
masterdf.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


> Now, let us drop the extra columns, to start with we do not want any retweets in our analysis, so I am truncating my dataframe to choose only those rows where the value in the retweetwd_status_id is null

In [22]:
masterdf = masterdf[masterdf['retweeted_status_id'].isnull()]
# let us verify our job was done by looking at info
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      1994 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     1994 non-null object
source                        1994 non-null object
text                          1994 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1994 non-null object
rating_numerator              1994 non-null int64
rating_denominator            1994 non-null int64
name                          1994 non-null object
doggo                         1994 non-null object
floofer                       1994 non-null object
pupper                        1994 non-null object
puppo                         1994 non-null object
favorite_count                1994 non

> Now let us drop the extra columns, p2, p2_conf, p2_dog, p3, p3_conf,p3_dog, retweeted_status_user_id, retweeted_status_timestamp, retweeted_status_id, img_num, in_reply_to_status_id, in_reply_to_user_id

In [23]:
masterdf.drop(["p2", "p2_conf", "p2_dog", "p3", "p3_conf","p3_dog", "retweeted_status_user_id", "retweeted_status_timestamp", "retweeted_status_id", "img_num", "in_reply_to_status_id", "in_reply_to_user_id"], axis=1, inplace=True)
# let us confirm the changes have been made
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 18 columns):
tweet_id              1994 non-null int64
timestamp             1994 non-null object
source                1994 non-null object
text                  1994 non-null object
expanded_urls         1994 non-null object
rating_numerator      1994 non-null int64
rating_denominator    1994 non-null int64
name                  1994 non-null object
doggo                 1994 non-null object
floofer               1994 non-null object
pupper                1994 non-null object
puppo                 1994 non-null object
favorite_count        1994 non-null int64
retweet_count         1994 non-null int64
jpg_url               1994 non-null object
p1                    1994 non-null object
p1_conf               1994 non-null float64
p1_dog                1994 non-null bool
dtypes: bool(1), float64(1), int64(5), object(11)
memory usage: 282.4+ KB


> Now let us put all dog stage columns together into one column by the name Dog_stage instead of multiple columns

In [24]:
#first lets join all the column values
masterdf['Dog_stage'] = masterdf['puppo']+masterdf['doggo']+masterdf['floofer']+masterdf['pupper']
#column values are strings with only the below combinations and they need to be chnaged accordingly
masterdf['Dog_stage'].replace("NoneNoneNoneNone","None ", inplace=True)
masterdf['Dog_stage'].replace("puppoNoneNoneNone","puppo", inplace=True)
masterdf['Dog_stage'].replace("NonedoggoNoneNone","doggo", inplace=True)
masterdf['Dog_stage'].replace("NoneNoneflooferNone","floofer", inplace=True)
masterdf['Dog_stage'].replace("NoneNoneNonepupper","pupper", inplace=True)
# let us drop extra columns
masterdf.drop(['puppo','doggo','floofer','pupper'], axis=1, inplace=True)
# let us verify our plan is successful
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id              1994 non-null int64
timestamp             1994 non-null object
source                1994 non-null object
text                  1994 non-null object
expanded_urls         1994 non-null object
rating_numerator      1994 non-null int64
rating_denominator    1994 non-null int64
name                  1994 non-null object
favorite_count        1994 non-null int64
retweet_count         1994 non-null int64
jpg_url               1994 non-null object
p1                    1994 non-null object
p1_conf               1994 non-null float64
p1_dog                1994 non-null bool
Dog_stage             1994 non-null object
dtypes: bool(1), float64(1), int64(5), object(8)
memory usage: 235.6+ KB


In [25]:
#now let us deal with the timestamp. I intend to use the date to find how tweets have changed over time so 
# I will keep year and month together and hour of the day in another column named Hour_of_day 
masterdf['timestamp']=pd.to_datetime(masterdf['timestamp'])
masterdf['hour_of_day']=masterdf.timestamp.apply(lambda x: x.hour)
masterdf['timestamp']=masterdf.timestamp.apply(lambda x:x.strftime("%m/%Y/%d"))
masterdf['timestamp']=masterdf.timestamp.apply(lambda x:datetime.datetime.strptime(x, "%m/%Y/%d"))
# let us verify our changes were made
print(masterdf.info())
masterdf.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 16 columns):
tweet_id              1994 non-null int64
timestamp             1994 non-null datetime64[ns]
source                1994 non-null object
text                  1994 non-null object
expanded_urls         1994 non-null object
rating_numerator      1994 non-null int64
rating_denominator    1994 non-null int64
name                  1994 non-null object
favorite_count        1994 non-null int64
retweet_count         1994 non-null int64
jpg_url               1994 non-null object
p1                    1994 non-null object
p1_conf               1994 non-null float64
p1_dog                1994 non-null bool
Dog_stage             1994 non-null object
hour_of_day           1994 non-null int64
dtypes: bool(1), datetime64[ns](1), float64(1), int64(6), object(7)
memory usage: 251.2+ KB
None


,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,favorite_count,retweet_count,jpg_url,p1,p1_conf,p1_dog,Dog_stage,hour_of_day
0,892420643555336193,2017-08-01,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,38826,8594,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,orange,0.097049,False,None,16
1,892177421306343426,2017-08-01,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,33259,6316,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,Chihuahua,0.323581,True,None,0


In [26]:
# Next, lets fix the rating_denominator valuecounts since they should all ideally be 10
masterdf.rating_denominator.value_counts()
# a majority of ratings are infact 10, but 1994-1976=18 are still different. We could look into each entry and try to correct
# it manually or we could use regex to extract the correct ratings. I want to experiment with regex so I am going to use that
# another way would have been truncating our data to only those ratings that had denominator 10 but then we might be losing 
# some interesting information so I did not choose that path
# Using the resources provided in this degree on regex at https://regexone.com/
# pandas.extract method at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html 
masterdf['rating']=masterdf.text.str.extract(r'(\d+)/10', expand=True)
# some ratings were not successfully extracted so dropping those columns now
masterdf.dropna(subset=['rating'])
# earlier numerator and denominator columns are not needed anymore
masterdf.drop(["rating_numerator","rating_denominator"], axis=1, inplace=True)
# now lets verify all is fine
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id          1994 non-null int64
timestamp         1994 non-null datetime64[ns]
source            1994 non-null object
text              1994 non-null object
expanded_urls     1994 non-null object
name              1994 non-null object
favorite_count    1994 non-null int64
retweet_count     1994 non-null int64
jpg_url           1994 non-null object
p1                1994 non-null object
p1_conf           1994 non-null float64
p1_dog            1994 non-null bool
Dog_stage         1994 non-null object
hour_of_day       1994 non-null int64
rating            1981 non-null object
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(8)
memory usage: 235.6+ KB


In [27]:
# seems like we still have some ratings that havent filled up, so lets drop those columns again
masterdf = masterdf[masterdf['rating'].notnull()]
# let us verify the clean up was successful
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1981 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id          1981 non-null int64
timestamp         1981 non-null datetime64[ns]
source            1981 non-null object
text              1981 non-null object
expanded_urls     1981 non-null object
name              1981 non-null object
favorite_count    1981 non-null int64
retweet_count     1981 non-null int64
jpg_url           1981 non-null object
p1                1981 non-null object
p1_conf           1981 non-null float64
p1_dog            1981 non-null bool
Dog_stage         1981 non-null object
hour_of_day       1981 non-null int64
rating            1981 non-null object
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(8)
memory usage: 234.1+ KB


> Now let us fix the source column. It has a lot of extra info of urls that is not needed

In [28]:
masterdf['source'] = masterdf.source.str.extract(r'>(.*)<', expand=True)
# let us verify the changes were successful
masterdf.source.value_counts()

Twitter for iPhone    1942
Twitter Web Client      28
TweetDeck               11
Name: source, dtype: int64

In [29]:
# lets see what more needs to be done
masterdf.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1981 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id          1981 non-null int64
timestamp         1981 non-null datetime64[ns]
source            1981 non-null object
text              1981 non-null object
expanded_urls     1981 non-null object
name              1981 non-null object
favorite_count    1981 non-null int64
retweet_count     1981 non-null int64
jpg_url           1981 non-null object
p1                1981 non-null object
p1_conf           1981 non-null float64
p1_dog            1981 non-null bool
Dog_stage         1981 non-null object
hour_of_day       1981 non-null int64
rating            1981 non-null object
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(8)
memory usage: 234.1+ KB


In [30]:
# lets change the datatype for tweet_id to str
masterdf['tweet_id']=masterdf['tweet_id'].astype(str)
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1981 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id          1981 non-null object
timestamp         1981 non-null datetime64[ns]
source            1981 non-null object
text              1981 non-null object
expanded_urls     1981 non-null object
name              1981 non-null object
favorite_count    1981 non-null int64
retweet_count     1981 non-null int64
jpg_url           1981 non-null object
p1                1981 non-null object
p1_conf           1981 non-null float64
p1_dog            1981 non-null bool
Dog_stage         1981 non-null object
hour_of_day       1981 non-null int64
rating            1981 non-null object
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(9)
memory usage: 234.1+ KB


In [31]:
# the column expanded_urls is not needed, 
# I dont intend to use name as well and text so lets drop those as well. Then we will rename other columns
masterdf.drop(['text','name','expanded_urls'], axis=1, inplace=True)
# lets verify that the changes were infact made
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1981 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id          1981 non-null object
timestamp         1981 non-null datetime64[ns]
source            1981 non-null object
favorite_count    1981 non-null int64
retweet_count     1981 non-null int64
jpg_url           1981 non-null object
p1                1981 non-null object
p1_conf           1981 non-null float64
p1_dog            1981 non-null bool
Dog_stage         1981 non-null object
hour_of_day       1981 non-null int64
rating            1981 non-null object
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 187.7+ KB


In [32]:
# lets rename the columns to something more explicit
masterdf=masterdf.rename(columns={'timestamp':'Date_of_tweet','p1':'Prediction_Dog_Breed','jpg_url':'Dog_image'})
# I wont be using p1_conf and p1_dog so dropping those columns as well
masterdf.drop(['p1_conf','p1_dog'], axis=1, inplace=True)
# lets verify our changes were infact made
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1981 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id                1981 non-null object
Date_of_tweet           1981 non-null datetime64[ns]
source                  1981 non-null object
favorite_count          1981 non-null int64
retweet_count           1981 non-null int64
Dog_image               1981 non-null object
Prediction_Dog_Breed    1981 non-null object
Dog_stage               1981 non-null object
hour_of_day             1981 non-null int64
rating                  1981 non-null object
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 170.2+ KB


In [33]:
# This completes our cleaning of the dataset. 
# I am going to store this clean dataframe into a file and then analyse the data in the next step. 
masterdf.to_csv('twitter_archive_master.csv', index=False)